In [ ]:
# libraries
# !pip install openai
# !pip install langchain
# !pip install eodhd
# !pip install config

In [4]:
import re
import requests
import pandas as pd
from eodhd import APIClient     # EOD Hitorical DATA로 data API 제공
from langchain.chains import LLMChain
from langchain.prompts import PromptTemplate
from langchain.chat_models import ChatOpenAI
import keyring

In [6]:
api_key = keyring.get_password('eodhd', 'ahn283')
api = APIClient(api_key)

In [7]:
resp = api.financial_news(s = 'MSFT', from_date='2024-01-01', to_date='2024-03-20', limit=100)
df = pd.DataFrame(resp)     # transfrom json output into dataframe
df.tail()

,date,title,content,link,symbols,tags,sentiment
95,2024-03-14T07:03:28+00:00,"Oracle adds generative AI features to finance,...",By Stephen Nellis\n\n(Reuters) - Oracle on Thu...,https://finance.yahoo.com/news/oracle-adds-gen...,"[0QYP.IL, MSF.BE, MSF.DU, MSF.F, MSF.HM, MSF.M...",[],"{'polarity': 0.612, 'neg': 0.014, 'neu': 0.954..."
96,2024-03-14T00:00:00+00:00,7 Delightful Dividend Stocks to Put Some Sprin...,The best dividend stocks can be counted on to ...,https://investorplace.com/market360/2024/03/7-...,"[AAPL.US, AVGO.US, JPM.US, LLY.US, MA.US, MSFT...",[],"{'polarity': 0.856, 'neg': 0, 'neu': 0.732, 'p..."
97,2024-03-14T00:00:00+00:00,3 Virtual Reality Stocks to Buy as the VR Mark...,While you might not personally be a fan of dig...,https://investorplace.com/2024/03/3-virtual-re...,"[META.US, MSFT.US, U.US]",[],"{'polarity': 0.399, 'neg': 0.081, 'neu': 0.792..."
98,2024-03-14T00:00:00+00:00,"\nAnders Bylund | Mar 14, 2024\n","\nAnders Bylund | Mar 14, 2024\n",https://www.fool.com/terms/t/turnaround/,"[AAPL.US, MSFT.US, NFLX.US]",[],"{'polarity': 0, 'neg': 0, 'neu': 1, 'pos': 0}"
99,2024-03-14T00:00:00+00:00,"\nDanny Vena | Mar 14, 2024\n",Wall Street sees big upside for the company's ...,https://www.fool.com/investing/2024/03/14/why-...,[MSFT.US],[],"{'polarity': 0.477, 'neg': 0, 'neu': 0.78, 'po..."


In [10]:
# preprocessing of text data
def clean_text(text):
    cleaned_text = re.sub(r'\s+', ' ', text)
    return cleaned_text.strip()

In [11]:
# apply preprocessing to all rows
df['content'] = df['content'].apply(clean_text)

In [12]:
# Instantiate ChatOpenAI instance
llm = ChatOpenAI(
    model='gpt-4',
    openai_api_key=keyring.get_password('openai', 'key_for_mac'),
    temperature=0
)

In [13]:
template = """
Identify the sentiment towards the MSFT stocks from the news article , where the sentiment score should be from -10 to +10 where -10 being the most negative and +10 being the most positve , and 0 being neutral
Also give the proper explanation for your answers and how would it effect the prices of different stocks
Article : {statement}
"""

# Create prompts using Langchain PromptTemplate 
prompt = PromptTemplate(template=template, input_variables=["statement"])
llm_chain = LLMChain(prompt=prompt, llm=llm)

In [14]:
print(llm_chain.run(df['content'][13]))

The sentiment towards MSFT stocks from this news article is +8. 

The article is highly positive about Microsoft's recent actions and future prospects. The hiring of Mustafa Suleyman, a well-known figure in the AI industry, is likely to strengthen Microsoft's leading position in the booming AI industry. This is a strategic move that could potentially lead to the development of innovative products and services, thereby increasing the company's competitive advantage. 

Furthermore, Microsoft's partnerships with OpenAI and Mistral, as well as its significant investments in AI, indicate the company's commitment to staying at the forefront of technological advancements. This could lead to increased investor confidence in the company's ability to generate future profits, which could positively impact the stock price.

However, the sentiment is not a full +10 as the article also mentions the need to limit potential perils of AI, indicating that there are still challenges and risks associated 